# Сбор маркетинговых кампаний с Facebook

In [60]:
from facebook_business.api import FacebookAdsApi
from facebook_business.exceptions import FacebookRequestError
from facebook_business.adobjects.adaccount import AdAccount
from facebook_business.adobjects.adreportrun import AdReportRun
from facebook_business.adobjects.adsinsights import AdsInsights
from facebook_business.adobjects.campaign import Campaign
from facebook_business.adobjects.adset import AdSet
from facebook_business.adobjects.adaccountuser import AdAccountUser as AdUser
from facebook_business import adobjects
from matplotlib import pyplot as plt
from pandas import DataFrame
import time
import datetime
import csv

In [65]:
my_app_id = '35607******9'
my_app_secret = 'f37******a66bca6'
my_access_token = 'EAAFD****ZC8ZD' #

In [66]:
FacebookAdsApi.init(my_app_id, my_app_secret, my_access_token)  # авторизуемся

In [ ]:
me = AdUser(fbid='me')    # добавляем пользователя
#me = AdAccount('https://www.facebook.com/pg/bbandburgers/')
my_accounts = list(me.get_ad_accounts())     # данные по рекламным аккаунтам

In [44]:
my_accounts

[<AdAccount> {
     "account_id": "1242079709464075",
     "id": "act_1242079709464075"
 }]

In [ ]:
my_account = my_accounts[0]
campaigns = my_account.get_campaigns()  # информация о кампаниях

In [ ]:
my_account.api_get(fields=[AdAccount.Field.amount_spent])  # получим затраты
print(int(my_account[AdAccount.Field.amount_spent])/100)   # обрубить копейки

In [57]:
# id кампании, количество кликов, затрат и просмотров
fields = [
    AdsInsights.Field.campaign_id,
    AdsInsights.Field.clicks,
    AdsInsights.Field.spend,
    AdsInsights.Field.impressions]

count = 0

In [32]:
def wait_for_async_job(async_job):
    global count
    async_job = async_job.api_get()
    while async_job[AdReportRun.Field.async_status] != 'Job Completed' or async_job[
        AdReportRun.Field.async_percent_completion] < 100:
        time.sleep(2)
        async_job = async_job.api_get()
    else:
        print("Job " + str(count) + " completed")
        count += 1
    return async_job.get_result(params={"limit": 1000})

def get_insights(account, date_preset='last_3d'):
    account = AdAccount(account["id"])
    i_async_job = account.get_insights(
        params={
            'level': 'ad',
            'date_preset': date_preset,
            'time_increment': 1},
            fields=fields,
            is_async=True)
    results = [dict(item) for item in wait_for_async_job(i_async_job)]
    return results

In [ ]:
elem_insights = []
insights_lists = []
date_preset = 'last_year'
for elem in my_accounts:
    elem_insights = get_insights(elem, date_preset)
    insights_lists.append(elem_insights)


In [49]:
insight_campaign_id_list = []
insight_clicks_list = []
insight_spend_list = []
insight_impressions_list = []
insight_date_start_list = []
insight_date_stop_list = []
for elem1 in insights_lists:
    for elem2 in elem1:
        insight_campaign_id_list.append(int(elem2['campaign_id']))
        insight_clicks_list.append(int(elem2['clicks']))
        insight_spend_list.append(float(elem2['spend']))
        insight_impressions_list.append(int(elem2['impressions']))
        insight_date_start_list.append(elem2['date_start'])
        insight_date_stop_list.append(elem2['date_stop'])


In [50]:
df = DataFrame()
df['campaign_id'] = insight_campaign_id_list
df['clicks'] = insight_clicks_list
df['spend'] = insight_spend_list
df['impressions'] = insight_impressions_list
df['date_start'] = insight_date_start_list
df['date_stop'] = insight_date_stop_list


In [51]:
df

,campaign_id,clicks,spend,impressions,date_start,date_stop


In [ ]:
# Запись в таблицу SQL

def ins(row):
    db_campaign = row['campaign_id']
    db_clicks = row['clicks']
    db_spend = row['spend']
    db_impressions = row['impressions']
    db_date_start = row['date_start']
    db_date_stop = row['date_stop']

    sql = "INSERT INTO likes ( campaign_id, clicks, spend, impressions, date_start, date_stop ) VALUES ( %s )"
    val = [(db_campaign, db_clicks, db_spend, db_impressions, db_date_start, db_date_stop)]

    cursor = connection.cursor()
    cursor.executemany(sql, val)
    connection.commit()
    return 'ok'

df.apply(ins)

# DDL

In [68]:
# Подключение к серверу MySQL. connection возвращает объект подключения.

import mysql.connector
from mysql.connector import Error

def create_connection(host_name, user_name, user_password):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password
        )
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

connection = create_connection("localhost", "root", "")

The error '2003: Can't connect to MySQL server on 'localhost:3306' (10061 Подключение не установлено, т.к. конечный компьютер отверг запрос на подключение)' occurred


In [ ]:
# Создание базы данных

def create_database(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("Database created successfully")
    except Error as e:
        print(f"The error '{e}' occurred")
        
create_database_query = "CREATE DATABASE facebook_insights"
create_database(connection, create_database_query)

In [ ]:
# Подключение к базе данных

def create_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

connection = create_connection("localhost", "root", "", "facebook_insights")

In [ ]:
# Создадим таблицу в БД facebook_insights

def execute_query(connection, query):
     cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

create_users_table = """
CREATE TABLE IF NOT EXISTS users (
  campaign_id INT NOT NULL, 
  clicks INT, 
  spend INT, 
  impressions INT, 
  date_start DATE,
  date_stop DATE
) ENGINE = InnoDB
"""

execute_query(connection, create_users_table)